# Initial Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectFromModel
from collections import defaultdict
from imblearn.over_sampling import SVMSMOTE
from imblearn.under_sampling import RandomUnderSampler
import scipy.stats as stats

# Preprocessing

In [2]:
# Import dataset
df = pd.read_csv("./kieranFeatures_1-31_21-Jan-2025_avgof3_rawSA.csv").replace("#NAME?", np.nan).replace([np.inf, -np.inf], np.nan)
df.iloc[:, 781] = df.iloc[:, 781].astype(np.float64)
df.iloc[:, 782] = df.iloc[:, 782].astype(np.float64)

# Create Low vs High Columns
df["Lv_1_Lo"] = (df["SA1"] < 5).astype(np.bool_)
df["Lv_2_Lo"] = (df["SA2"] < 5).astype(np.bool_)
df["Lv_3_Lo"] = (df["SA3"] < 5).astype(np.bool_)
df["Tot_Lo"] = (df["SAtotal"] < 5).astype(np.bool_)



#
# Impute Data
#

# Impute missing values with mean of column
for col in df.columns:
    df[col] = df[col].fillna(value = df[col].mean())



#
# Split up Dataset
#
ids = df["ID"].astype(np.uint8)
trial_nums = df["trialNum"].astype(np.uint8)
predictors_df = df.drop(columns = ["ID", "trialNum", "SA1", "SA2", "SA3", "SAtotal", "Lv_1_Lo", "Lv_2_Lo", "Lv_3_Lo", "Tot_Lo"]).astype(np.float64)
outcomes_df = df[["SA1", "SA2", "SA3", "SAtotal", "Lv_1_Lo", "Lv_2_Lo", "Lv_3_Lo", "Tot_Lo"]].astype(np.float64)



#
# Free Up Memory
#
del col, df

/tmp/ipykernel_12439/3838981478.py:2: DtypeWarning: Columns (781,782) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./kieranFeatures_1-31_21-Jan-2025_avgof3_rawSA.csv").replace("#NAME?", np.nan).replace([np.inf, -np.inf], np.nan)
/tmp/ipykernel_12439/3838981478.py:20: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(value = df[col].mean())


In [15]:
df.select_dtypes(include = "object", exclude = "float")["EEG_frpcv_o1_version11"]

0          -1.84785
1        252.438729
2          4.902291
3          0.622102
4           0.36862
           ...     
295    -3.402921835
296     5.298826105
297    -3.224659795
298    -2.598417765
299     0.008975581
Name: EEG_frpcv_o1_version11, Length: 300, dtype: object

In [18]:
type(df.loc[0, "EEG_frpcv_o1_version11"])

float